# Problem 1 - Data is in CSV files

## Attempt 1 - Pandas

In [ ]:
import pandas as pd

Upload the dataset. For this demo we'll load just one dataset but you could load all the datasets

In [ ]:
# Also load call_types for better definitions
call_types = pd.read_csv(
     "assets/pd_cfs_calltypes_datasd.csv",
     keep_default_na=False
)


### Some Helper Scripts we built to Make sure that data was consistent on upload.

In [ ]:
def convert_floats_to_ints(val):
    try:
        return int(float(val))

    except (TypeError, ValueError):
        return val


def strip_priority(val):
    try:
        return convert_floats_to_ints(val[0])

    except (TypeError, ValueError, IndexError):
        return val



In [ ]:
csv_file = "assets/pd_calls_for_service_2021_datasd.csv"
df = pd.read_csv(
                csv_file,
                keep_default_na=False,
                parse_dates=["date_time"],
            )
df.head()
len(df)

## Let's See What We can Do

In [ ]:
df_w_call_type = pd.merge(df, call_types[['call_type','description']], on="call_type", how="left").fillna('').drop_duplicates('incident_num')

## The last line is a little ugly let's explain

`pd.merge` - combine the our `call types` data with our actual stop data.

`.fillna` - creates consistency with the rest of our empty fields (replacing NaN/NA values)

`.drop_duplicates` - not sure why duplicates are created but this is common and I discovered that removing the duplicates on the `incident_num` (unique for each call) seems to resolve the issue.


## Can we make sense of this data?

In [ ]:
# we're goign to create a sorting help function

def sort_group(group):
    return sorted(group, key=lambda x:len(x[1]), reverse=True)

df_w_call_type["beat"] = df_w_call_type["beat"].apply(convert_floats_to_ints)
df_w_call_type["priority"] = df_w_call_type["priority"].apply(strip_priority)

In [ ]:
#  Calls per Beat in 2021

grouped_beats = sort_group(df_w_call_type.groupby('beat'))

for beat, group in grouped_beats[:5]:
    num_of_calls = len(group)
    print(f"{beat=}, {num_of_calls=}")

In [ ]:
# Beat 523 Calls

beat_523 = df_w_call_type[df_w_call_type.beat==523]
beat_523.head()

In [ ]:
# Busiest Intersections for Beat 523

intersections = beat_523[beat_523.address_road_intersecting != '']
grouped_beats = sort_group(intersections.groupby(['address_road_primary', 'address_road_intersecting']))

for intersection, group in grouped_beats[:5]:
    num_of_calls = len(group)
    print(f"{intersection=}, {num_of_calls=}")

In [ ]:
beat_523['priority'].value_counts().plot(kind="bar")

In [ ]:
beat_523['description'].value_counts()[:5].plot(kind="bar")

# This is great but a little narrow 
## Let's Load Data into Elasticsearch

## Why Though?

- Offload Storage and Processing to Server
- Visualization Tools (Kibana Lens)

## Let's Look at our Traditional Option

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

import hashlib
import os

# Load Client
client = Elasticsearch(
    hosts=[os.environ["ELASTICSEARCH_HOST"]],  # for local instance
)

# upload data to Elasticsearch (because we have the data loaded into pandas we can just export the dataframe to json)

bulk(client=client, index='test_2021_index', actions=df.to_dict('records'))    

## Cool Data is in and we can work with it in Kibana

# But what did all the work (pandas)

`# upload data to Elasticsearch (because we have the data loaded into pandas we can just export the dataframe to json)`

## How do we work with the data in python if we start with the data in Elasticsearch?

In [ ]:
results = client.search(
    index="test_2021_index",
    body={
        "query": {
            "match_all": {}
       }
    })['hits']['hits']

new_df = pd.DataFrame([x['_source'] for x in results])
new_df.head()

len(new_df)

## Can we make changes to this data?

[Let's check this](http://jays-mac-mini-3.local:5601/app/dashboards#/view/b5c671c0-3e71-11eb-b5a2-43487ca632dc?_g=(filters:!(),refreshInterval:(pause:!t,value:0),time:(from:now-8y,to:now))&_a=(description:'',filters:!(),fullScreenMode:!f,options:(hidePanelTitles:!f,useMargins:!t),panels:!((embeddableConfig:(),gridData:(h:26,i:'8ad5d19f-c5e0-41c9-89b4-03c6a1e8b09a',w:12,x:0,y:0),id:'13720c30-3f35-11eb-b5a2-43487ca632dc',panelIndex:'8ad5d19f-c5e0-41c9-89b4-03c6a1e8b09a',type:visualization,version:'7.10.0'),(embeddableConfig:(hiddenLayers:!(),isLayerTOCOpen:!f,mapCenter:(lat:32.82517,lon:-116.91398,zoom:8.11),openTOCDetails:!()),gridData:(h:14,i:'40a2aa29-3a4f-4bc6-b5fc-65f97926ede3',w:32,x:12,y:0),id:'4f55ff10-3f33-11eb-b5a2-43487ca632dc',panelIndex:'40a2aa29-3a4f-4bc6-b5fc-65f97926ede3',type:map,version:'7.10.0'),(embeddableConfig:(),gridData:(h:12,i:ea97ada1-6535-42f9-8881-2d616d70a134,w:10,x:23,y:14),id:a939c2c0-3e9b-11eb-b5a2-43487ca632dc,panelIndex:ea97ada1-6535-42f9-8881-2d616d70a134,type:visualization,version:'7.10.0'),(embeddableConfig:(),gridData:(h:12,i:'9fdc5b9f-5f85-4aef-a066-f74c1df2b5cf',w:11,x:33,y:14),id:'1460f8c0-3e74-11eb-b5a2-43487ca632dc',panelIndex:'9fdc5b9f-5f85-4aef-a066-f74c1df2b5cf',type:visualization,version:'7.10.0'),(embeddableConfig:(),gridData:(h:13,i:e18eb168-dc4f-4bb7-83a3-ccb262d34f85,w:12,x:0,y:26),id:c3b65300-3f33-11eb-b5a2-43487ca632dc,panelIndex:e18eb168-dc4f-4bb7-83a3-ccb262d34f85,type:visualization,version:'7.10.0'),(embeddableConfig:(),gridData:(h:13,i:'3c0c0e0f-b33c-4971-a8bf-8a2a7fc2c63a',w:32,x:12,y:26),id:f68052b0-3e9b-11eb-b5a2-43487ca632dc,panelIndex:'3c0c0e0f-b33c-4971-a8bf-8a2a7fc2c63a',type:lens,version:'7.10.0'),(embeddableConfig:(),gridData:(h:11,i:'75a467b0-b343-44dd-a547-d144a9ef629e',w:44,x:0,y:39),id:'7f47ea40-3e74-11eb-b5a2-43487ca632dc',panelIndex:'75a467b0-b343-44dd-a547-d144a9ef629e',type:lens,version:'7.10.0'),(embeddableConfig:(),gridData:(h:15,i:'01918722-445d-4dbd-ace3-0b5f488afd5b',w:44,x:0,y:50),id:fd2eca90-3e75-11eb-b5a2-43487ca632dc,panelIndex:'01918722-445d-4dbd-ace3-0b5f488afd5b',type:lens,version:'7.10.0')),query:(language:kuery,query:''),timeRestore:!f,title:'PD%20Calls%20Overtime',viewMode:edit))

Yes, but...
In order to do this we need to make the changes in the dataframe and then re-upload.

## Let's talk about [Eland](https://eland.readthedocs.io/en/7.10.1b1/#)

## Eland is Elasticsearch data in a Dataframe view

It's like

```python
results = client.search(
    body={
        "size": 3,
        "query": {
            "match_all": {}
       }
    })['hits']['hits']

new_df = pd.DataFrame([x['_source'] for x in results])
```

but `new_df` is connected to your Elasticsearch instance.

In [ ]:
import eland

# all things Elasticsearch use 'es_' as a prefix

edf = eland.DataFrame(es_client=client, es_index_pattern="test_2021_index")

In [ ]:
ebeat_523 = edf[edf.beat==523]
ebeat_523

In [ ]:
# Compared to our original
beat_523

In [ ]:
beat_523['description'].value_counts()[:5].plot(kind="bar")

In [ ]:
beat_523['description'].value_counts()[:5].plot(kind="bar")

In [ ]:
sd_pd_data = eland.DataFrame(es_client=client, es_index_pattern="pd_calls_for_service_*")
sd_pd_data['description'].value_counts()[:5].plot(kind="bar")